
# Hyperparameter Tuning

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


#Load the data

In [ ]:
import pandas as pd

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv' ).dropna().drop(['customerID'], axis=1, )
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


#Clean the data if necessary (it will be)

In [ ]:
df.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [ ]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [ ]:
df['Churn'].map({'No': 0, 'Yes': 1}).value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [ ]:
df['gender'].value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

In [ ]:
df['Partner'].map({'No': 0, 'Yes': 1}).value_counts()

0    3641
1    3402
Name: Partner, dtype: int64

In [ ]:
df['TotalCharges'].str.replace(' ', '0').astype(float)

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7043, dtype: float64

In [ ]:
def wrangle(dataframe):
  df = dataframe.copy()

  #ONE-HOT ENCODE:
  #1.INTERNET SERVICE:
  df['fiber_optic'] = df['InternetService'] == 'Fiber_Optic'
  df['dsl'] = df['InternetService'] == 'DSL'

  #2. Contract Types:
  df['Contract_mtm'] = df['Contract'] == 'Month_to_Mont'
  df['Contract_2yrs'] = df['Contract'] == 'Two_years'
  df['Contract_1yr'] = df['Contract'] == 'One_year'
  df = df.drop(columns='Contract')

  #3.Payment Method:
  df['Payment_echeck'] = df['PaymentMethod'] == 'Electronic_check'
  df['Payment_Mailcheck'] = df['PaymentMethod'] == 'Mailed_check'
  df['Payment_bank'] = df['PaymentMethod'] == 'Bank_Transfer'
  df['Payment_credCard']  = df['PaymentMethod'] == 'Credit_card'
  df = df.drop(columns='PaymentMethod')

  #Categorical Columns:
  Bin_Columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
                 'Churn']

  for column in Bin_Columns:
    df[column] = df [column].map({
            'Male': 0,
            'Female': 1,
            'No': 0,
            'Yes': 1,
            'No phone service': 0,
            'Fiber optic': 1,
            'DSL': 1,
            'No internet service': 0,
    })

  #Nan 
  df['TotalCharges'] = df['TotalCharges'].str.replace(' ', '0').astype(float)
  
  for column in df.columns:
    df[column] = df[column].astype(float)

  return df

In [ ]:
Data_clean = wrangle(df)
Data_clean

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,fiber_optic,dsl,Contract_mtm,Contract_2yrs,Contract_1yr,Payment_echeck,Payment_Mailcheck,Payment_bank,Payment_credCard
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,29.85,29.85,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,34.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,56.95,1889.50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,53.85,108.15,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,45.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,42.30,1840.75,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,70.70,151.65,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.0,0.0,1.0,1.0,24.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,84.80,1990.50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7039,1.0,0.0,1.0,1.0,72.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,103.20,7362.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7040,1.0,0.0,1.0,1.0,11.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,29.60,346.45,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7041,0.0,1.0,1.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,74.40,306.60,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
Data_clean['Churn'].value_counts(normalize=True)

0.0    0.73463
1.0    0.26537
Name: Churn, dtype: float64

#Normalize

In [ ]:
from tensorflow.keras.utils import normalize

target = 'Churn'
features = Data_clean.drop(columns=target).columns

X = Data_clean[features].values
y = Data_clean[target].values

print('X', X.shape, type(X))
print('y', y.shape, type(y))

X (7043, 26) <class 'numpy.ndarray'>
y (7043,) <class 'numpy.ndarray'>


#Create and fit a baseline Keras MLP model to the data.

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/wrappers

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
def create_model(additional_layers=0, 
                 nodes_per_layer=26, 
                 activation_per_layer='relu',
                 loss_function='binary_crossentropy', 
                 optimizer='adam'):
  
    model = Sequential()

    model.add(Dense(nodes_per_layer, 
                    activation='relu', 
                    input_dim=26))
    
    for _ in range(additional_layers):
        model.add(Dense(nodes_per_layer, 
                        activation=activation_per_layer))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=loss_function, 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    
    return model

In [ ]:
#Implement scikit and keras
model = KerasClassifier(build_fn=create_model, verbose=1)
results = model.fit(X, y, epochs=10, 
                    validation_split=0.2, 
                    verbose=1)

Epoch 1/10
177/177 [==============================] - 0s 3ms/step - loss: 6.0640 - accuracy: 0.7208 - val_loss: 0.8404 - val_accuracy: 0.7800
Epoch 2/10
177/177 [==============================] - 0s 2ms/step - loss: 0.6699 - accuracy: 0.7506 - val_loss: 0.7275 - val_accuracy: 0.7814
Epoch 3/10
177/177 [==============================] - 0s 2ms/step - loss: 0.7184 - accuracy: 0.7529 - val_loss: 0.6942 - val_accuracy: 0.6629
Epoch 4/10
177/177 [==============================] - 0s 2ms/step - loss: 0.5975 - accuracy: 0.7661 - val_loss: 1.4389 - val_accuracy: 0.5628
Epoch 5/10
177/177 [==============================] - 0s 2ms/step - loss: 0.6483 - accuracy: 0.7648 - val_loss: 1.3931 - val_accuracy: 0.7842
Epoch 6/10
177/177 [==============================] - 0s 2ms/step - loss: 0.6236 - accuracy: 0.7717 - val_loss: 0.4628 - val_accuracy: 0.7771
Epoch 7/10
177/177 [==============================] - 0s 2ms/step - loss: 0.5086 - accuracy: 0.7776 - val_loss: 0.8210 - val_accuracy: 0.7878
Epoch 

#Hyperparameter tune (at least) the following parameters:

-batch_size

-training epochs

-optimizer

-learning rate (if applicable to optimizer)

-momentum (if applicable to optimizer)

-activation functions

-network weight initialization

-dropout regularization

-number of neurons in the hidden layer

In [ ]:
#Machine learning Quick reference book
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import  EarlyStopping

stop = EarlyStopping(monitor='accuracy', 
                     min_delta=0.001, 
                     patience=5)
#Number of Nodes
params = {
    'nodes_per_layer': [20, 42, 78, 100]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
)

grid.fit(X, y, callbacks=[stop], 
         epochs=10, validation_split=0.2, verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Epoch 1/10
141/141 [==============================] - 0s 2ms/step - loss: 8.0140 - accuracy: 0.6974 - val_loss: 0.6473 - val_accuracy: 0.7524
Epoch 2/10
141/141 [==============================] - 0s 2ms/step - loss: 0.5882 - accuracy: 0.7244 - val_loss: 0.6897 - val_accuracy: 0.7711
Epoch 3/10
141/141 [==============================] - 0s 2ms/step - loss: 0.5995 - accuracy: 0.7273 - val_loss: 0.5143 - val_accuracy: 0.7817
Epoch 4/10
141/141 [==============================] - 0s 2ms/step - loss: 0.5467 - accuracy: 0.7451 - val_loss: 0.5234 - val_accuracy: 0.7187
Epoch 5/10
141/141 [==============================] - 0s 2ms/step - loss: 0.5456 - accuracy: 0.7624 - val_loss: 0.5086 - val_accuracy: 0.7950
Epoch 6/10
141/141 [==============================] - 0s 2ms/step - loss: 0.5219 - accuracy: 0.7695 - val_loss: 0.5700 - val_accuracy: 0.7915
Epoch 7/10
141/141 [==============================] - 0s 2ms/step - loss: 0.5182 - accuracy: 0.7759 - val_loss: 0.5085 - val_accuracy: 0.7968
Epoch 

In [ ]:
grid.score(X,y)

221/221 [==============================] - 0s 1ms/step - loss: 0.4731 - accuracy: 0.7862


0.7861706614494324

In [ ]:
#Additional layers
params = {
    'nodes_per_layer': [50],
    'additional_layers': [10,8,4,0]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    scoring='accuracy',
    n_jobs=1,
    cv=5,
)

grid.fit(X, y, callbacks=[stop], 
         epochs=10,
         validation_split=0.2, verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Epoch 1/10
141/141 [==============================] - 1s 4ms/step - loss: 0.6648 - accuracy: 0.7400 - val_loss: 0.5333 - val_accuracy: 0.7720
Epoch 2/10
141/141 [==============================] - 0s 3ms/step - loss: 0.5397 - accuracy: 0.7668 - val_loss: 0.5377 - val_accuracy: 0.7755
Epoch 3/10
141/141 [==============================] - 0s 3ms/step - loss: 0.5410 - accuracy: 0.7573 - val_loss: 0.6158 - val_accuracy: 0.7746
Epoch 4/10
141/141 [==============================] - 0s 3ms/step - loss: 0.5264 - accuracy: 0.7681 - val_loss: 0.5615 - val_accuracy: 0.7764
Epoch 5/10
141/141 [==============================] - 0s 3ms/step - loss: 0.5074 - accuracy: 0.7715 - val_loss: 0.4727 - val_accuracy: 0.7826
Epoch 6/10
141/141 [==============================] - 0s 3ms/step - loss: 0.4900 - accuracy: 0.7726 - val_loss: 0.4641 - val_accuracy: 0.7897
Epoch 7/10
141/141 [==============================] - 0s 3ms/step - loss: 0.4866 - accuracy: 0.7695 - val_loss: 0.4797 - val_accuracy: 0.7799
Epoch 

In [ ]:
#Batch Size:
params = {
    'nodes_per_layer': [50],
    'additional_layers': [4],
    'batch_size': [10, 30, 60, 100],
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    scoring='accuracy',
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         epochs=10,
         validation_split=0.2, verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 1.2825 - accuracy: 0.6982 - val_loss: 1.8035 - val_accuracy: 0.7294
Epoch 2/10
451/451 [==============================] - 1s 2ms/step - loss: 0.7423 - accuracy: 0.7335 - val_loss: 0.5501 - val_accuracy: 0.7853
Epoch 3/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5597 - accuracy: 0.7639 - val_loss: 0.4869 - val_accuracy: 0.7808
Epoch 4/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5357 - accuracy: 0.7675 - val_loss: 0.4765 - val_accuracy: 0.7853
Epoch 5/10
451/451 [==============================] - 1s 2ms/step - loss: 0.4993 - accuracy: 0.7706 - val_loss: 0.4709 - val_accuracy: 0.7933
Epoch 6/10
451/451 [==============================] - 1s 2ms/step - loss: 0.4870 - accuracy: 0.7757 - val_loss: 0.4730 - val_accuracy: 0.7826
Epoch 7/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5038 - accuracy: 0.7681 - val_loss: 0.5076 - val_accuracy: 0.7826
Epoch 8/10
451/45

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.6min finished


564/564 [==============================] - 1s 2ms/step - loss: 1.6186 - accuracy: 0.6924 - val_loss: 1.0616 - val_accuracy: 0.7743
Epoch 2/10
564/564 [==============================] - 1s 2ms/step - loss: 0.7591 - accuracy: 0.7309 - val_loss: 0.5455 - val_accuracy: 0.7750
Epoch 3/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5389 - accuracy: 0.7691 - val_loss: 0.4710 - val_accuracy: 0.7970
Epoch 4/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5045 - accuracy: 0.7716 - val_loss: 0.4708 - val_accuracy: 0.7850
Epoch 5/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5416 - accuracy: 0.7664 - val_loss: 0.4781 - val_accuracy: 0.7729
Epoch 6/10
564/564 [==============================] - 1s 2ms/step - loss: 0.4973 - accuracy: 0.7767 - val_loss: 0.4846 - val_accuracy: 0.7906
Epoch 7/10
564/564 [==============================] - 1s 2ms/step - loss: 0.4814 - accuracy: 0.7836 - val_loss: 0.4803 - val_accuracy: 0.7878
Epoch 8/10
564/56

In [ ]:
#Optimizing:
params = {
    'nodes_per_layer': [50],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'optimizer': ['adam', 'adadelta'
                  'nadam', 'rmsprop', 'ftrl']
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         epochs=10,
         validation_split=0.2, verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 0.7979 - accuracy: 0.7140 - val_loss: 0.8047 - val_accuracy: 0.7728
Epoch 2/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5699 - accuracy: 0.7550 - val_loss: 0.5294 - val_accuracy: 0.7782
Epoch 3/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5241 - accuracy: 0.7699 - val_loss: 0.4922 - val_accuracy: 0.7906
Epoch 4/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5127 - accuracy: 0.7763 - val_loss: 0.4734 - val_accuracy: 0.7844
Epoch 5/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5069 - accuracy: 0.7701 - val_loss: 0.4788 - val_accuracy: 0.7844
Epoch 6/10
451/451 [==============================] - 1s 2ms/step - loss: 0.4988 - accuracy: 0.7755 - val_loss: 0.4980 - val_accuracy: 0.7773
Epoch 7/10
451/451 [==============================] - 1s 2ms/step - loss: 0.4718 - accuracy: 0.7788 - val_loss: 0.4696 - val_accuracy: 0.7853
Epoch 8/10
451/45

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unknown optimizer: adadeltanadam

  FitFailedWarning)


Epoch 1/10
451/451 [==============================] - 1s 2ms/step - loss: 1.5512 - accuracy: 0.6796 - val_loss: 0.6799 - val_accuracy: 0.7693
Epoch 2/10
451/451 [==============================] - 1s 2ms/step - loss: 0.7012 - accuracy: 0.7344 - val_loss: 0.5241 - val_accuracy: 0.7835
Epoch 3/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5848 - accuracy: 0.7570 - val_loss: 0.4799 - val_accuracy: 0.7844
Epoch 4/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5575 - accuracy: 0.7570 - val_loss: 0.5117 - val_accuracy: 0.7817
Epoch 5/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7582 - val_loss: 0.7844 - val_accuracy: 0.5439
Epoch 6/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5886 - accuracy: 0.7495 - val_loss: 0.4920 - val_accuracy: 0.7924
Epoch 7/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5527 - accuracy: 0.7653 - val_loss: 0.9762 - val_accuracy: 0.7808
Epoch 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  2.4min finished


564/564 [==============================] - 1s 2ms/step - loss: 1.6491 - accuracy: 0.7009 - val_loss: 0.7702 - val_accuracy: 0.7786
Epoch 2/10
564/564 [==============================] - 1s 2ms/step - loss: 0.6135 - accuracy: 0.7503 - val_loss: 1.2933 - val_accuracy: 0.7771
Epoch 3/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5713 - accuracy: 0.7618 - val_loss: 0.8068 - val_accuracy: 0.5706
Epoch 4/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5274 - accuracy: 0.7696 - val_loss: 0.4779 - val_accuracy: 0.7928
Epoch 5/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5238 - accuracy: 0.7661 - val_loss: 0.4646 - val_accuracy: 0.7814
Epoch 6/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5183 - accuracy: 0.7733 - val_loss: 0.7474 - val_accuracy: 0.5884
Epoch 7/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5035 - accuracy: 0.7772 - val_loss: 0.4806 - val_accuracy: 0.7835
Epoch 8/10
564/56

In [ ]:
from tensorflow.keras.optimizers import Adam

def create_model(additional_layers=0, 
                 nodes_per_layer=26, 
                 activation_per_layer='relu',
                 loss_function='binary_crossentropy', 
                 learning_rate=0.001):
  
    model2 = Sequential()

    model2.add(Dense(nodes_per_layer, 
                    activation='relu', 
                    input_dim=26))
    
    for _ in range(additional_layers):
        model2.add(Dense(nodes_per_layer, 
                        activation=activation_per_layer))
    model2.add(Dense(1, activation='sigmoid'))

    #Optimiser model
    optimizer = Adam(learning_rate=learning_rate)

    model2.compile(loss=loss_function, 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    
    return model2

In [ ]:
#Learning rate for Adam Optimizer:

params = {
    'nodes_per_layer': [50],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001, 0.01, 0.1] 
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         epochs=10,
         validation_split=0.2, verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 1.5810 - accuracy: 0.6954 - val_loss: 1.0533 - val_accuracy: 0.7755
Epoch 2/10
451/451 [==============================] - 1s 2ms/step - loss: 0.7937 - accuracy: 0.7346 - val_loss: 0.7218 - val_accuracy: 0.7933
Epoch 3/10
451/451 [==============================] - 1s 2ms/step - loss: 0.7471 - accuracy: 0.7471 - val_loss: 0.5453 - val_accuracy: 0.7915
Epoch 4/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7637 - val_loss: 0.5057 - val_accuracy: 0.7702
Epoch 5/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5337 - accuracy: 0.7590 - val_loss: 0.4925 - val_accuracy: 0.7808
Epoch 6/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5260 - accuracy: 0.7628 - val_loss: 0.4758 - val_accuracy: 0.7870
Epoch 7/10
451/451 [==============================] - 1s 2ms/step - loss: 0.5141 - accuracy: 0.7684 - val_loss: 0.4473 - val_accuracy: 0.7906
Epoch 8/10
451/45

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.2min finished


564/564 [==============================] - 1s 2ms/step - loss: 1.1021 - accuracy: 0.7071 - val_loss: 0.7059 - val_accuracy: 0.7686
Epoch 2/10
564/564 [==============================] - 1s 2ms/step - loss: 0.6084 - accuracy: 0.7561 - val_loss: 0.5171 - val_accuracy: 0.7850
Epoch 3/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5563 - accuracy: 0.7650 - val_loss: 0.5015 - val_accuracy: 0.7828
Epoch 4/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5185 - accuracy: 0.7730 - val_loss: 0.5052 - val_accuracy: 0.7764
Epoch 5/10
564/564 [==============================] - 1s 2ms/step - loss: 0.6111 - accuracy: 0.7599 - val_loss: 0.5036 - val_accuracy: 0.7743
Epoch 6/10
564/564 [==============================] - 1s 2ms/step - loss: 0.5213 - accuracy: 0.7751 - val_loss: 0.4771 - val_accuracy: 0.7736
Epoch 7/10
564/564 [==============================] - 1s 2ms/step - loss: 0.4903 - accuracy: 0.7799 - val_loss: 0.4714 - val_accuracy: 0.7835
Epoch 8/10
564/56

In [ ]:
#Adam Optimizer
params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001, 0.01, 0.1]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         validation_split=0.2, 
         verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 1.5586 - accuracy: 0.6763 - val_loss: 0.6689 - val_accuracy: 0.7817
Epoch 2/100
451/451 [==============================] - 1s 2ms/step - loss: 0.7356 - accuracy: 0.7397 - val_loss: 0.6622 - val_accuracy: 0.7862
Epoch 3/100
451/451 [==============================] - 1s 2ms/step - loss: 0.6287 - accuracy: 0.7519 - val_loss: 0.4718 - val_accuracy: 0.7941
Epoch 4/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5646 - accuracy: 0.7617 - val_loss: 0.4695 - val_accuracy: 0.7799
Epoch 5/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5609 - accuracy: 0.7637 - val_loss: 0.4862 - val_accuracy: 0.7906
Epoch 6/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5091 - accuracy: 0.7786 - val_loss: 0.5228 - val_accuracy: 0.7853
Epoch 7/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5360 - accuracy: 0.7684 - val_loss: 0.8883 - val_accuracy: 0.5377
Epoch 8/100

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.5min finished


564/564 [==============================] - 1s 2ms/step - loss: 1.0688 - accuracy: 0.7055 - val_loss: 0.9213 - val_accuracy: 0.7757
Epoch 2/100
564/564 [==============================] - 1s 2ms/step - loss: 0.6211 - accuracy: 0.7536 - val_loss: 0.4785 - val_accuracy: 0.7842
Epoch 3/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5404 - accuracy: 0.7668 - val_loss: 0.5376 - val_accuracy: 0.7835
Epoch 4/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5170 - accuracy: 0.7705 - val_loss: 0.4667 - val_accuracy: 0.7857
Epoch 5/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5214 - accuracy: 0.7717 - val_loss: 0.5216 - val_accuracy: 0.7892
Epoch 6/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5009 - accuracy: 0.7788 - val_loss: 0.5070 - val_accuracy: 0.7346
Epoch 7/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4861 - accuracy: 0.7790 - val_loss: 0.4546 - val_accuracy: 0.7913
Epoch 8/100

In [ ]:
from tensorflow.keras.optimizers import Adam

#Compile the model
def create_model(additional_layers=0, 
                 nodes_per_layer=26, 
                 activation_per_layer='relu',
                 loss_function='binary_crossentropy', 
                 learning_rate=0.001, 
                 network_weights_init='uniform'):
  
    model4 = Sequential()

    model4.add(Dense(nodes_per_layer, 
                    activation='relu', 
                    input_dim=26, 
                    kernel_initializer=network_weights_init))
    
    for _ in range(additional_layers):
        model4.add(Dense(nodes_per_layer, 
                        activation=activation_per_layer, 
                        kernel_initializer=network_weights_init))
    model4.add(Dense(1, activation='sigmoid', 
                    kernel_initializer=network_weights_init))

    optimizer = Adam(learning_rate=learning_rate)
    
    model4.compile(loss=loss_function, 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    
    return model4

In [ ]:
params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001],
    'network_weights_init': ['uniform', 
                             'zero', 
                             'glorot_normal']
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         validation_split=0.2, verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 0.5622 - accuracy: 0.7537 - val_loss: 0.5375 - val_accuracy: 0.7853
Epoch 2/100
451/451 [==============================] - 1s 2ms/step - loss: 0.4986 - accuracy: 0.7759 - val_loss: 0.4571 - val_accuracy: 0.7950
Epoch 3/100
451/451 [==============================] - 1s 2ms/step - loss: 0.4755 - accuracy: 0.7783 - val_loss: 0.4627 - val_accuracy: 0.7862
Epoch 4/100
451/451 [==============================] - 1s 2ms/step - loss: 0.4631 - accuracy: 0.7886 - val_loss: 0.4655 - val_accuracy: 0.7844
Epoch 5/100
451/451 [==============================] - 1s 2ms/step - loss: 0.4604 - accuracy: 0.7881 - val_loss: 0.4469 - val_accuracy: 0.7826
Epoch 6/100
451/451 [==============================] - 1s 2ms/step - loss: 0.4597 - accuracy: 0.7857 - val_loss: 0.4427 - val_accuracy: 0.8012
Epoch 7/100
451/451 [==============================] - 1s 2ms/step - loss: 0.4604 - accuracy: 0.7852 - val_loss: 0.4430 - val_accuracy: 0.7933
Epoch 8/100

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.9min finished


564/564 [==============================] - 1s 2ms/step - loss: 0.5586 - accuracy: 0.7522 - val_loss: 0.5254 - val_accuracy: 0.7864
Epoch 2/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4896 - accuracy: 0.7812 - val_loss: 0.4699 - val_accuracy: 0.7899
Epoch 3/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4662 - accuracy: 0.7819 - val_loss: 0.4622 - val_accuracy: 0.7921
Epoch 4/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4598 - accuracy: 0.7861 - val_loss: 0.4543 - val_accuracy: 0.7878
Epoch 5/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4593 - accuracy: 0.7870 - val_loss: 0.4491 - val_accuracy: 0.7913
Epoch 6/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4532 - accuracy: 0.7925 - val_loss: 0.4472 - val_accuracy: 0.7942
Epoch 7/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4548 - accuracy: 0.7907 - val_loss: 0.4449 - val_accuracy: 0.7913
Epoch 8/100

In [ ]:
#activation functions

params = {
    'nodes_per_layer': [50],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001],
    'network_weights_init': ['glorot_normal'],
    'activation_per_layer': ['relu', 'sigmoid', 'softmax'],
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         validation_split=0.2, 
         verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 1.3402 - accuracy: 0.6903 - val_loss: 0.5565 - val_accuracy: 0.7347
Epoch 2/100
451/451 [==============================] - 1s 2ms/step - loss: 0.7136 - accuracy: 0.7351 - val_loss: 0.5260 - val_accuracy: 0.7906
Epoch 3/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5815 - accuracy: 0.7588 - val_loss: 0.5060 - val_accuracy: 0.7773
Epoch 4/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5262 - accuracy: 0.7695 - val_loss: 0.4500 - val_accuracy: 0.7906
Epoch 5/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5277 - accuracy: 0.7610 - val_loss: 0.4679 - val_accuracy: 0.7959
Epoch 6/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5139 - accuracy: 0.7659 - val_loss: 0.4945 - val_accuracy: 0.7906
Epoch 7/100
451/451 [==============================] - 1s 2ms/step - loss: 0.5089 - accuracy: 0.7695 - val_loss: 0.4677 - val_accuracy: 0.7888
Epoch 8/100

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.4min finished


564/564 [==============================] - 1s 2ms/step - loss: 0.8950 - accuracy: 0.7114 - val_loss: 0.5249 - val_accuracy: 0.7807
Epoch 2/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5601 - accuracy: 0.7639 - val_loss: 0.4866 - val_accuracy: 0.7835
Epoch 3/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7639 - val_loss: 0.4940 - val_accuracy: 0.7722
Epoch 4/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5054 - accuracy: 0.7744 - val_loss: 0.5456 - val_accuracy: 0.7793
Epoch 5/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4849 - accuracy: 0.7764 - val_loss: 0.4825 - val_accuracy: 0.7899
Epoch 6/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4901 - accuracy: 0.7765 - val_loss: 0.4628 - val_accuracy: 0.7899
Epoch 7/100
564/564 [==============================] - 1s 2ms/step - loss: 0.4744 - accuracy: 0.7831 - val_loss: 0.4854 - val_accuracy: 0.7814
Epoch 8/100

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

#Compile the model and dropout rate
def create_model(additional_layers=0, 
                 nodes_per_layer=26, 
                 activation_per_layer='relu',
                 loss_function='binary_crossentropy', 
                 learning_rate=0.001,
                 network_weights_init='uniform', 
                 dropout_rate=0.1):
  
    model6 = Sequential()

    model6.add(Dense(nodes_per_layer, 
                    activation=activation_per_layer, input_dim=26, 
                    kernel_initializer=network_weights_init))
    
    for _ in range(additional_layers):
        model6.add(Dropout(rate=dropout_rate))
        model6.add(Dense(nodes_per_layer, 
                        activation=activation_per_layer, 
                        kernel_initializer=network_weights_init))
    model6.add(Dense(1, activation='sigmoid', 
                    kernel_initializer=network_weights_init))

    optimizer = Adam(learning_rate=learning_rate)
    
    model6.compile(loss=loss_function, 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    
    return model6

In [ ]:
#Dropout

params = {
    'nodes_per_layer': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001],
    'network_weights_init': ['glorot_normal'],
    'activation_per_layer': ['relu'],
    'dropout_rate': [0.1, 0.2, 0.3]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], 
         validation_split=0.2, 
         verbose=1)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


451/451 [==============================] - 1s 2ms/step - loss: 5.4537 - accuracy: 0.6319 - val_loss: 1.3152 - val_accuracy: 0.6903
Epoch 2/100
451/451 [==============================] - 1s 1ms/step - loss: 1.2793 - accuracy: 0.6869 - val_loss: 1.2597 - val_accuracy: 0.7365
Epoch 3/100
451/451 [==============================] - 1s 1ms/step - loss: 1.1935 - accuracy: 0.7049 - val_loss: 2.5946 - val_accuracy: 0.4330
Epoch 4/100
451/451 [==============================] - 1s 1ms/step - loss: 0.8949 - accuracy: 0.7224 - val_loss: 1.5370 - val_accuracy: 0.7595
Epoch 5/100
451/451 [==============================] - 1s 1ms/step - loss: 0.8834 - accuracy: 0.7428 - val_loss: 0.5070 - val_accuracy: 0.7711
Epoch 6/100
451/451 [==============================] - 1s 1ms/step - loss: 1.1117 - accuracy: 0.7397 - val_loss: 0.5742 - val_accuracy: 0.7702
Epoch 7/100
451/451 [==============================] - 1s 1ms/step - loss: 0.8514 - accuracy: 0.7511 - val_loss: 0.5499 - val_accuracy: 0.7959
Epoch 8/100

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.8min finished


564/564 [==============================] - 1s 2ms/step - loss: 0.7532 - accuracy: 0.7359 - val_loss: 0.6206 - val_accuracy: 0.7296
Epoch 2/100
564/564 [==============================] - 1s 1ms/step - loss: 0.6008 - accuracy: 0.7359 - val_loss: 0.5932 - val_accuracy: 0.7296
Epoch 3/100
564/564 [==============================] - 1s 1ms/step - loss: 0.5835 - accuracy: 0.7359 - val_loss: 0.5854 - val_accuracy: 0.7296
Epoch 4/100
564/564 [==============================] - 1s 2ms/step - loss: 0.5785 - accuracy: 0.7359 - val_loss: 0.5835 - val_accuracy: 0.7296
Epoch 5/100
564/564 [==============================] - 1s 1ms/step - loss: 0.5775 - accuracy: 0.7359 - val_loss: 0.5832 - val_accuracy: 0.7296
Epoch 6/100
564/564 [==============================] - 1s 1ms/step - loss: 0.5772 - accuracy: 0.7359 - val_loss: 0.5832 - val_accuracy: 0.7296

Best score: 0.7851788878440857
Best params: {'activation_per_layer': 'relu', 'batch_size': 10, 'dropout_rate': 0.3, 'epochs': 100, 'learning_rate': 0.001


- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?